AkHQ: http://localhost:8082  
Spark-Master http://localhost:8083  
Spark-Worker-1 http://localhost:8084

https://github.com/aehrc/pathling/tree/issue/452/lib/python#python-api-for-pathling

# Try Connect To Test FHIR Kafka

In [1]:
!python --version
!yes | pip uninstall pathling
!yes | pip install pathling==5.0.3.dev1

Python 3.10.4
Found existing installation: pathling 5.0.3.dev1
Uninstalling pathling-5.0.3.dev1:
  Would remove:
    /opt/conda/lib/python3.10/site-packages/pathling-5.0.3.dev1.dist-info/*
    /opt/conda/lib/python3.10/site-packages/pathling/*
    /opt/conda/share/pathling/examples/data/bundles/Bennett146_Swaniawski813_704c9750-f6e6-473b-ee83-fbd48e07fe3f.json
    /opt/conda/share/pathling/examples/data/bundles/Dino214_Parisian75_40d82b80-b682-cd8b-da6d-396809878641.json
    /opt/conda/share/pathling/examples/data/resources/Condition.ndjson
    /opt/conda/share/pathling/examples/data/resources/Patient.ndjson
    /opt/conda/share/pathling/examples/encode_bundle.py
    /opt/conda/share/pathling/examples/encode_resources.py
Proceed (Y/n)?   Successfully uninstalled pathling-5.0.3.dev1
yes: standard output: Broken pipe
  Using cached pathling-5.0.3.dev1-py2.py3-none-any.whl (19.6 MB)
yes: standard output: Broken pipe


In [2]:
import os
os.environ

environ{'SHELL': '/bin/bash',
        'HOSTNAME': '356c87a855a5',
        'LANGUAGE': 'en_US.UTF-8',
        'SPARK_OPTS': '--driver-java-options=-Xms1024M --driver-java-options=-Xmx4096M --driver-java-options=-Dlog4j.logLevel=info',
        'NB_UID': '1000',
        'PWD': '/home/jovyan',
        'PYSPARK_SUBMIT_ARGS': '     --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1,au.csiro.pathling:encoders:5.0.3-SNAPSHOT     --repositories https://oss.sonatype.org/content/repositories/snapshots     pyspark-shell',
        'HOME': '/home/jovyan',
        'LANG': 'en_US.UTF-8',
        'NB_GID': '100',
        'XDG_CACHE_HOME': '/home/jovyan/.cache/',
        'APACHE_SPARK_VERSION': '3.2.1',
        'PYTHONPATH': '/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip:/usr/local/spark/python:',
        'HADOOP_VERSION': '3.2',
        'SHLVL': '0',
        'CONDA_DIR': '/opt/conda',
        'SPARK_HOME': '/usr/local/spark',
        'SPARK_CONF_DIR': '/usr/local/spark/conf',
        'NB_U

In [3]:
# for debugging
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell"

In [3]:
appName = "Kafka, Spark and FHIR Data"
master = "spark://spark-master:7077"
#master = "local[*]"
kafka_topic = "fhir.post-gateway-kdb"

In [4]:
from pyspark.sql import SparkSession
    
spark = SparkSession.builder \
        .appName(appName) \
        .master(master) \
        .config('spark.ui.port','0') \
        .getOrCreate()

In [5]:
# https://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
df = spark \
  .readStream  \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka1:19092") \
  .option("subscribe", kafka_topic) \
  .option("startingOffsets", "earliest") \
  .load()

In [6]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
query = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
          .writeStream \
          .queryName("gettable") \
          .format("memory") \
          .start()

# close connection after 30 seconds
query.awaitTermination(30)

False

In [8]:
kafka_data = spark.sql("select * from gettable")
kafka_data.show()
type(kafka_data)

+-------------------+--------------------+
|                key|               value|
+-------------------+--------------------+
|    Patient/patient|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Condition/condition|{"resourceType": ...|
|Procedure/procedure|{"resourceType": ...|
|Encounter/encounter|{"resourceType": ...|
+-------------------+--------------------+



pyspark.sql.dataframe.DataFrame

In [9]:
pd_df = kafka_data.toPandas()
pd_df

,key,value
0,Patient/patient,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
1,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
2,Condition/condition,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."
3,Procedure/procedure,"{""resourceType"": ""Bundle"",""meta"": {""security"":..."
4,Encounter/encounter,"{""resourceType"": ""Bundle"", ""meta"": {""security""..."


# Bring Pathling into the game

In [10]:
from pathling import PathlingContext

ptl = PathlingContext.create(spark)

In [12]:
# readme-example
#n_bundles = spark.read.text('/home/jovyan/pathling/lib/python/examples/data/bundles/', wholetext=True)
#n_bundles
#type(n_bundles)

In [11]:
kafka_data.select("value")

DataFrame[value: string]

In [13]:
patients = ptl.encodeBundle(kafka_data.select("value"), 'Patient')
encounter = ptl.encodeBundle(kafka_data.select("value"), 'Encounter')
condition = ptl.encodeBundle(kafka_data.select("value"), 'Condition')

In [14]:
patients.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [15]:
patients.select("id", "birthDate", "gender", "address.postalCode").toPandas()

,id,birthDate,gender,postalCode
0,patient,1990-01-01,female,[12345]


In [16]:
encounter.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [18]:
encounter.select("id", "subject.reference", "serviceType.coding.code", "period.start", "period.end").toPandas()

,id,reference,code,start,end
0,encounter,Patient/patient,"[2200, 2200]",2021-11-09T05:41:53+00:00,None


In [19]:
condition.printSchema()

root
 |-- id: string (nullable = true)
 |-- id_versioned: string (nullable = true)
 |-- meta: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- versionId: string (nullable = true)
 |    |-- versionId_versioned: string (nullable = true)
 |    |-- lastUpdated: timestamp (nullable = true)
 |    |-- source: string (nullable = true)
 |    |-- profile: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- security: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- system: string (nullable = true)
 |    |    |    |-- version: string (nullable = true)
 |    |    |    |-- code: string (nullable = true)
 |    |    |    |-- display: string (nullable = true)
 |    |    |    |-- userSelected: boolean (nullable = true)
 |    |-- tag: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable

In [18]:
condition.select("id", "encounter.reference", "code.coding.code").toPandas()

,id,reference,code
0,condition,Encounter/encounter,[H57.0]
1,condition,Encounter/encounter,[H57.0]


In [19]:
from pyspark.sql.functions import regexp_replace, col

pt1 = patients.select("id", "birthDate", "gender", "address.postalCode")

enc1 = encounter \
    .select( \
        col("id").alias("encounter_id"), \
        "subject.reference", \
        col("serviceType.coding.code").alias("servicetype_code"), \
        col("period.start").alias("period_start"), \
        col("period.end").alias("period_end") \
       ) \
    .withColumn("patient_id", regexp_replace("reference", "Patient/", ""))

cd1 = condition \
    .select( \
        "id", \
        "encounter.reference", \
        col("code.coding.code").alias("condition_code") \
       ) \
    .withColumn("cond_encounter_id", regexp_replace("reference", "Encounter/", ""))

In [20]:
join1 = enc1 \
    .join(pt1, enc1.patient_id == pt1.id)
join2 = join1 \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode") \
    .join(cd1, join1.encounter_id == cd1.cond_encounter_id) \
    .select("encounter_id", "servicetype_code", "period_start", "period_end", "patient_id", "birthDate", "gender", "postalCode", "condition_code")

In [21]:
join2.toPandas()

,encounter_id,servicetype_code,period_start,period_end,patient_id,birthDate,gender,postalCode,condition_code
0,encounter,"[2200, 2200]",2021-11-09T05:41:53+00:00,None,patient,1990-01-01,female,[12345],[H57.0]
1,encounter,"[2200, 2200]",2021-11-09T05:41:53+00:00,None,patient,1990-01-01,female,[12345],[H57.0]
